### 0a-Agents: Homework

#### Preparation:

First, we'll define a function that we will use when building our agents. 

It will generate fake weather data:

In [4]:
# Weather tool 

import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

#### Q1. Define function Description 

We want to use it as a tool for our agent, So we need to describe it: #

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves the current weather information for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to retrieve weather information."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [6]:
from chat_assistant import Tools, ChatAssistant, ChatInterface
from openai import OpenAI

# Tool 
tools = Tools() 

tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Provides current weather information for given city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city name we want weather information from'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [9]:
# Prompt Template
developer_prompt = """
You're a helpful AI Assistant. 

please provide a concise and precise answer. 

Always use Tools if you need, but never generate a answer without checking with available tools.
""".strip()


# chat interface
chat_interface = ChatInterface()

# OpenAI client 
client = OpenAI()


chat = ChatAssistant(
    tools=tools, 
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [10]:
chat.run()

Chat ended.


### Q2. Adding another Tool

Let's add another tool - a function that can add weather data to our database:

In [1]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Tool Description: `set_weather_tool`

In [10]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Adds new city and weather data into the database",
    "parameters": {
        "type" : "object", 
        "properties": {
            "city": {
                "type": "string", 
                "descrption": "city name to add in the database"
            },
            "temp": {
                "type": "number",
                "description": "asscoaited temperature to the city to add in the database"
            }
        }
    }
}

In [11]:
from chat_assistant import ChatAssistant, ChatInterface, Tools
import openai

# Tools 
tools = Tools()

# Adding new tool 
tools.add_tool(get_weather, get_weather_tool)
tools.add_tool(set_weather, set_weather_tool)

# get tools 
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Provides current weather information for given city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city name we want weather information from'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Adds new city and weather data into the database',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'descrption': 'city name to add in the database'},
    'temp': {'type': 'number',
     'description': 'asscoaited temperature to the city to add in the database'}}}}]

In [12]:
# Prompt Template
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()


# openai client 
client = openai.OpenAI()


# chat interface
chat_interface = ChatInterface()

# chat assistant 
chat = ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)


In [ ]:
chat.run()

### MCP: Model-Context Protocol


`MCP` Stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further: 

* A tool can export a list of functions it has. 

* When we include the tool to our Agent, we just need to include the link to the MCP Server.

### Q3. Install FastMCP

Let's install a library for MCP - `FastMCP`:

In [3]:
# pip install fastmcp

In [3]:
import fastmcp 

fastmcp.__version__

'2.9.2'

### Q4. Simple MCP Server 

A simple MCP server from the documentation looks like that: 

In [ ]:
# weather_server.py 
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

if __name__ == "__main__":
    mcp.run()

```bash
[07/16/25 12:00:13] INFO     Starting MCP server 'Demo 🚀' with transport 'stdio'      
```

Q. What do you have instead of <TODO>?

Answer: 'stdio' - Standard Input/Output

### Q5. Protocol

There are different ways to communicate with an MCP Server. Ours is currently running using standard input/output, which means that the client write something to `stdin` and read the answer using `stdout`. 

Our weather server is currently running: 

This is how we start communicating with it: 

* First, we send an initialization request -- this way, we register our client with the server: 

```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

We should get back something like that, which is an acknowledgement of the request: 

```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```

* Next, we reply back, confirming the initialization: 
 ```json
 {"jsonrpc": "2.0", "method": "notifications/initialized"}
 ```

We don't expect to get anything in response. 

* Now we can ask for a list of available methods (Tools): 

```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

* Let's ask the temperature in berlin: 

```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
```

Q. What did you get in response ?


Request: 
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "berlin"}}}
```

Response: 

```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"isError":false}}
```

### Q6. Client 

We typically don't interact with the server by copy-pasting commands in the terminal. 

In practice, we use an MCP Client. Let's implement it..

`FastMCP` also supports MCP Client: 

In [12]:
import weather_server
from fastmcp import Client 

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(f"Available tools: {tools}")

In [14]:
await main()

Available tools: [Tool(name='get_weather', description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, annotations=None), Tool(name='set_weather', description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, annotations=None)]
